# Cahn-Hilliard Equation Simulation in Python

This notebook simulates the Cahn-Hilliard equation in two dimensions. 

First, we'll set-up imports, model parameters, and folder for saving results. 

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from scipy.io import savemat

# System parameters
delta = 10  # nm
sigma = 1.0e-19  # J/nm^2
A = 3 * sigma / delta  # J/nm^3, Height of the barrier between two energy wells
K = delta**2 * A # J/nm
A = A * 10  # Need for speed?
D = 1e9  # nm^2/s

# Since d2fdphi2 = 2*A, the mobility M is constant
M = D / (2 * A)

# Time and simulation settings
dT = 1e-12
steps = int(1e3)
save_interval = steps/10

# Mesh grid
h = 1  # spacing, nm
N = 200  # Mesh size

# Create output directory based on current time
dir_name = time.strftime('%H-%M-%S')
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

### Define Cahn-Hilliard equation

In [6]:
# Define the Laplacian using periodic boundary conditions
def laplacian(u, h):
    return (np.roll(u, 1, axis=0) + np.roll(u, -1, axis=0) +
            np.roll(u, 1, axis=1) + np.roll(u, -1, axis=1) - 4*u) / h**2

## Experiment 1 

### Initial Condition

In [7]:
# Initial Condition
# phi = np.full((N, N), 0.1)
# phi(:,end/2+1:end,1) = 1; % Block of A and Block of B
phi = 0.5 - np.random.rand(N, N)

# savemat(os.path.join(dir_name, f't_0_{steps}Steps.mat'), {'phi': phi})

phi_max = np.max(phi)
phi_min = np.min(phi)

plt.figure()
plt.imshow(phi, origin='lower', cmap='viridis', vmin=phi_min, vmax=phi_max)
plt.title(f't = 0')
plt.colorbar()
plot_filename = os.path.join(dir_name, f't_0_{steps}Steps.png')
plt.savefig(plot_filename)
plt.close()

### Numerical Simulation

In [8]:
t_index = 0
volume_fraction = np.zeros(steps) 
interfacial_area = np.zeros(steps) 

# ----------------------
# Simulation Loop
# ----------------------
for step in range(steps):
    # Compute derivative of free energy
    dfdphi = A * (4 * phi**3 - 6 * phi**2 + 2 * phi)
    
    # Compute Laplacian of phi
    del2phi = laplacian(phi, h)
    
    # Compute the chemical potential term
    chemical_potential = dfdphi - 2 * K * del2phi
    
    # Compute the right-hand side
    RHS = M * laplacian(chemical_potential, h)
    
    # Update phi
    phi = phi + dT * RHS
    
    # Record volume fraction and interfacial area
    volume_fraction[t_index] = np.sum(phi > 0.7) / (N**2)
    interfacial_area[t_index] = np.sum(np.logical_and(phi > 0.3, phi < 0.7))
    
    # Save condition and plot every save_interval steps
    if (step + 1) % save_interval == 0:
        plt.figure()
        plt.imshow(phi, origin='lower', cmap='viridis', vmin=phi_min, vmax=phi_max)
        plt.title(f't = {step+1}e-12')
        plt.colorbar()
        plot_filename = os.path.join(dir_name, f't_{step+1}e-12_{steps}Steps.png')
        plt.savefig(plot_filename)
        plt.close()

        #savemat(os.path.join(dir_name, f't_{step+1}e-12_{steps}Steps.mat'), {'phi': phi})
        #savemat(os.path.join(dir_name, f't_{step+1}e-12_VolFrac_{steps}Steps.mat'), {'volume_fraction': volume_fraction[:t_index+1]})
        #savemat(os.path.join(dir_name, f't_{step+1}e-12_IntArea_{steps}Steps.mat'), {'interfacial_area': interfacial_area[:t_index+1]})
    
    t_index += 1

# Save final condition and plot if not already saved per 1e4 rule in loop
if (step + 1) % save_interval != 0:
    plt.figure()
    plt.imshow(phi, origin='lower', cmap='viridis', vmin=phi_min, vmax=phi_max)
    plt.title(f't = {steps}e-12 (Final)')
    plt.colorbar()
    plt.savefig(os.path.join(dir_name, f'Beta_{Beta}_Final_{steps}Steps.png'))
    plt.close()
    
    savemat(os.path.join(dir_name, f't_{step+1}e-12_{steps}Steps.mat'), {'phi': phi})
    #savemat(os.path.join(dir_name, f't_{step+1}e-12_VolFrac_{steps}Steps.mat'), {'volume_fraction': volume_fraction[:t_index+1]})
    #savemat(os.path.join(dir_name, f't_{step+1}e-12_IntArea_{steps}Steps.mat'), {'interfacial_area': interfacial_area[:t_index+1]})

print('Simulation complete. Results saved in directory:', dir_name)

Simulation complete. Results saved in directory: 23-53-55
